In [1]:
import numpy as np
import pandas as pd
import os
import sys
import pickle
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload
%matplotlib inline
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:80% !important; }</style>")) 

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

## Import PMF class

In [3]:
cwd = os.getcwd()
path = os.path.join(cwd, '..', 'src')
if not path in sys.path:
    sys.path.append(path)
del cwd, path

In [4]:
from MovieRecommender import PMF

## Depersist trained model

In [9]:
pmf = pickle.load(open('adam_pmf_model_trained.pkl', 'rb'))

## Load ratings data

In [10]:
cwd = os.getcwd()
ratings = pd.read_csv(os.path.join(cwd, "..", "data", "ratings.csv"))
ratings.drop('timestamp', axis=1, inplace=True)

## Build prediction request

In [11]:
adam_id = 138494

In [12]:
request = pd.DataFrame(ratings['movieId'].unique(), columns=['movieId'])

In [13]:
request['userId'] = adam_id

## Make predictions

In [14]:
preds = pmf.predict(request)

## Observe predictions

In [17]:
preds = preds.sort_values('prediction', ascending=False)

In [18]:
cwd = os.getcwd()
movies = pd.read_csv(os.path.join(cwd, "..", "data", "movies.csv"))

In [28]:
preds = pd.merge(preds, movies, on='movieId')

In [25]:
adam_profile = pd.read_csv('adam_profile.csv', index_col=0)

In [34]:
preds = pd.merge(preds, adam_profile, on='movieId', how='left')

In [46]:
havnet_seen_it_mask = preds['rating'].isnull()

In [51]:
preds[havent_seen_it_mask].head(30)

,movieId,userId,prediction,title,genres,rating
1,1201,138494,5.566099,"Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966)",Action|Adventure|Western,NaN
2,3527,138494,5.559449,Predator (1987),Action|Sci-Fi|Thriller,NaN
3,1221,138494,5.514657,"Godfather: Part II, The (1974)",Crime|Drama,NaN
4,1200,138494,5.509245,Aliens (1986),Action|Adventure|Horror|Sci-Fi,NaN
5,858,138494,5.466609,"Godfather, The (1972)",Crime|Drama,NaN
6,1196,138494,5.465391,Star Wars: Episode V - The Empire Strikes Back (1980),Action|Adventure|Sci-Fi,NaN
8,7502,138494,5.448516,Band of Brothers (2001),Action|Drama|War,NaN
9,57528,138494,5.423444,Rambo (Rambo 4) (2008),Action|Drama|Thriller|War,NaN
10,6,138494,5.422579,Heat (1995),Action|Crime|Thriller,NaN
11,1036,138494,5.412736,Die Hard (1988),Action|Crime|Thriller,NaN


## Persist top 2000 recommendations for use

In [50]:
preds[havent_seen_it_mask][['title', 'genres', 'prediction']][:2000].to_csv('movie_recommendations.csv', index=False)